# Libraries

In [26]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn import decomposition
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, f1_score
from sklearn.metrics import make_scorer, fbeta_score, precision_score, recall_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn import linear_model, neighbors
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV , train_test_split
import xgboost as xgb
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import sklearn.metrics as skm
import time
import shap
import joblib
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_row', 500)

import gc
from contextlib import contextmanager
import missingno as msno

## MLFlow setup

In [7]:
import mlflow
# Set the MLFlow tracking URI (local file storage)
mlflow.set_tracking_uri("./mlruns")

# Dataset

In [5]:
df = pd.read_csv('Home_credit_risk_data_modeling.csv')

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356251 entries, 0 to 356250
Columns: 798 entries, Unnamed: 0 to CC_COUNT
dtypes: bool(133), float64(606), int64(43), object(16)
memory usage: 1.8+ GB


In [19]:
df_train_sample = df.loc[~(df['TARGET'].isnull())]
df_test_sample = df.loc[df['TARGET'].isnull()]

In [20]:
df_train_sample[df_train_sample['TARGET'] == 1].shape

(24825, 798)

In [21]:
df_train_sample[df_train_sample['TARGET'] == 0].shape

(282682, 798)

# Modelisation

In [22]:
X = df_train_sample.drop(columns = ['TARGET']).copy()
y = df_train_sample['TARGET'].copy()

In [40]:
def kfold_lightgbm(df, num_folds, stratified = False, debug= False):
    # Divide in training/validation and test data
    train_df = df[df['TARGET'].notnull()]
    test_df = df[df['TARGET'].isnull()]
    print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
    del df
    gc.collect()
    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=1001)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=1001)
    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]

        # LightGBM parameters found by Bayesian optimization
        clf = LGBMClassifier(
            nthread=4,
            n_estimators=10000,
            learning_rate=0.02,
            num_leaves=34,
            colsample_bytree=0.9497036,
            subsample=0.8715623,
            max_depth=8,
            reg_alpha=0.041545473,
            reg_lambda=0.0735294,
            min_split_gain=0.0222415,
            min_child_weight=39.3259775,
            silent=-1 )

        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc')

        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
        sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()

    print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
    # Write submission file and plot feature importance
    if not debug:
        test_df['TARGET'] = sub_preds
        test_df[['SK_ID_CURR', 'TARGET']].to_csv(submission_file_name, index= False)
    display_importances(feature_importance_df)
    return feature_importance_df

In [41]:
features = kfold_lightgbm(df, 3, stratified = False, debug= False)

Starting LightGBM. Train shape: (307507, 798), test shape: (48744, 798)


ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: CC_NAME_CONTRACT_STATUS_Active_MIN: object, CC_NAME_CONTRACT_STATUS_Active_MAX: object, CC_NAME_CONTRACT_STATUS_Approved_MIN: object, CC_NAME_CONTRACT_STATUS_Approved_MAX: object, CC_NAME_CONTRACT_STATUS_Completed_MIN: object, CC_NAME_CONTRACT_STATUS_Completed_MAX: object, CC_NAME_CONTRACT_STATUS_Demand_MIN: object, CC_NAME_CONTRACT_STATUS_Demand_MAX: object, CC_NAME_CONTRACT_STATUS_Refused_MIN: object, CC_NAME_CONTRACT_STATUS_Refused_MAX: object, CC_NAME_CONTRACT_STATUS_Sent proposal_MIN: object, CC_NAME_CONTRACT_STATUS_Sent proposal_MAX: object, CC_NAME_CONTRACT_STATUS_Signed_MIN: object, CC_NAME_CONTRACT_STATUS_Signed_MAX: object, CC_NAME_CONTRACT_STATUS_nan_MIN: object, CC_NAME_CONTRACT_STATUS_nan_MAX: object